# Railway Delay Prediction Project

This notebook implements a machine learning pipeline for predicting railway delays using various ensemble models.

## Table of Contents
1. [Setup and Configuration](#setup)
2. [Data Loading](#data-loading)
3. [Exploratory Data Analysis](#eda)
4. [Feature Engineering](#feature-engineering)
5. [Model Training](#model-training)
6. [Model Evaluation](#evaluation)
7. [Results and Conclusions](#conclusions)

## Setup

# 1) Introduction — Background, Objectives, and Success Criteria

## 1.1 Problem Background

Train delays are a critical operational challenge that impacts multiple facets of railway systems:

- **Operational Impact**: Delays affect dispatching and scheduling operations, requiring real-time adjustments to maintain service reliability.
- **Passenger Experience**: Unpredictable delays diminish customer satisfaction and trust in rail services.
- **Resource Optimization**: Inefficient allocation of locomotives and carriages occurs when delays are not accurately anticipated.
- **Downstream Connections**: Delay propagation affects connecting services, creating cascading disruptions across the network.

Railway operational data exhibits unique characteristics:
- **Temporal Nature**: Data is inherently time-dependent, with strong temporal correlations between consecutive observations.
- **Chain Effect**: Delays often propagate along routes or across consecutive train services, creating interdependencies that must be captured in predictive models.

Understanding and predicting train delays enables proactive decision-making, improved resource allocation, and enhanced passenger communication.

---

## 1.2 Objectives

This project pursues two complementary analytical approaches to address the railway delay problem:

### A. Regression (Primary Task — Recommended)

**Objective**: Predict the magnitude of train delays measured in minutes.

- **Target Variable**: `DELAY_MINUTES` (or `TARGET` representing delay in minutes)
- **Output**: Continuous prediction of delay duration
- **Use Case**: Enables precise scheduling adjustments, resource reallocation, and detailed passenger information
- **Business Value**: Supports quantitative operational decisions and optimization algorithms

### B. Classification (Supporting Task — Optional)

**Objective**: Predict whether a train will be delayed based on a business-defined threshold.

- **Target Variable**: `IS_DELAYED` (binary: 0 = on-time, 1 = delayed)
- **Threshold Definition**: Commonly set at delays exceeding 5 or 10 minutes, depending on operational standards
- **Output**: 
  - Probability of delay occurrence
  - Binary delay classification label
- **Use Case**: Facilitates alerting systems, early warning mechanisms, and binary decision-making
- **Business Value**: Enables proactive interventions and resource mobilization

> **Recommendation**: Prioritize the **regression task** as the primary analysis to capture the full spectrum of delay severity. The classification task can complement this by providing binary alerts and supporting threshold-based operational policies.

---

## 1.3 Success Criteria

Clear evaluation metrics are essential to assess model performance and determine project success:

### Regression Task Metrics

- **Primary Metrics**:
  - **RMSE (Root Mean Squared Error)**: Penalizes large prediction errors, suitable when extreme delays are particularly costly
  - **MAE (Mean Absolute Error)**: Provides interpretable average prediction error in minutes
  
- **Evaluation Protocol**: 
  - Use **time-based test set** (chronological split) to simulate real-world deployment
  - Avoid random shuffling to preserve temporal integrity
  - Report metrics separately for different delay ranges (e.g., short delays < 10 min vs. long delays > 30 min)

### Classification Task Metrics

- **Primary Metrics**:
  - **PR-AUC (Precision-Recall Area Under Curve)**: Preferred over ROC-AUC when dealing with imbalanced delay classes
  - **F₂ Score**: Emphasizes recall over precision (β=2), appropriate when the business priority is to **detect as many delayed trains as possible**
  
- **Rationale for F₂ Score**:
  - The F_β score is defined as: 
    $$F_\beta = (1 + \beta^2) \cdot \frac{\text{precision} \cdot \text{recall}}{(\beta^2 \cdot \text{precision}) + \text{recall}}$$
  - With **β = 2**, recall is weighted **twice as much** as precision
  - This aligns with operational objectives where **missing a delayed train (false negative) is more costly** than falsely predicting a delay (false positive)
  
- **Additional Considerations**:
  - **Recall (Sensitivity)**: Critical for ensuring comprehensive delay detection
  - **Precision**: Balances false alarm rates to maintain operational credibility
  - **Threshold Optimization**: Adjust classification threshold to achieve desired precision-recall trade-off

---

### Summary of Success Criteria

| Task | Primary Metrics | Evaluation Approach | Target Performance |
|------|----------------|---------------------|-------------------|
| **Regression** | RMSE, MAE | Time-based test split | Minimize error; benchmark against baseline |
| **Classification** | PR-AUC, F₂ Score | Time-based test split | Maximize F₂ (prioritize recall) |

**Note**: Success will be measured not only by model performance metrics but also by the model's ability to generalize to future unseen data and provide actionable insights for railway operations.

---

# 2) Data Description — Overview & Metadata

## 2.1 Dataset Overview
- Each row represents an event (e.g., one train at one station or one journey segment).
- **Temporal columns**: scheduled/actual arrival and departure times.
- **Identifiers**: `TRAIN_ID`, `ROUTE_ID`, `STATION_ID`, `TRAIN_NUMBER`.
- **Contextual features**: weather conditions, holidays, time slots, operators, vehicle types.
- **Target labels**: 
  - Regression → `DELAY_MINUTES` (or `DELAY_ARRIVAL`)
  - Classification → `IS_DELAYED`

## 2.2 Data Dictionary (Summary Required)
A comprehensive data dictionary should include:
- Column name, description, data type, missing value percentage, example values
- **Level of measurement**: Nominal / Ordinal / Interval / Ratio — guides encoding and scaling choices

## 2.3 Basic Statistics
- Mean, median, standard deviation, IQR for numerical features
- Check for outliers and right-skewed distributions (consider `log1p` transformation when appropriate)

---

# 3) Data Preprocessing — Cleaning, Integration, Transformation, Split

## 3.1 Data Cleaning
- **Missing values**: 
  - Numeric → median imputation
  - Categorical → most frequent value or 'Unknown' category
  - Datetime parsing errors → NaT handling
- **Noisy data / outliers**: 
  - Preserve negative values (early arrivals) or clip based on business rules (e.g., clip delay < 0 → 0)
  - Apply winsorization for extreme values

## 3.2 Integration & Redundancy
- When merging multiple sources (schedule, actual, weather), ensure proper entity matching and avoid duplicate keys
- Analyze feature correlation to remove redundant variables

## 3.3 Transformation
- **Scaling**: StandardScaler for linear/SVM/kNN models; tree-based models don't require scaling
- **Encoding**: 
  - OneHotEncoder (with `handle_unknown='ignore'`)
  - Target encoding or hashing for high-cardinality features (careful to avoid leakage)
- **Target transformation**: Apply `log1p` if target distribution is right-skewed (consider metrics in log-space)

## 3.4 Time-Aware Split
- **No shuffling**: train = past, test = future
- Use `TimeSeriesSplit` for cross-validation to respect temporal ordering

---

# 4) EDA — Distributions, Temporal Patterns, Correlations

- **Univariate**: Histograms and boxplots of target variable (and log-transformed target)
- **Bivariate**: Delay patterns by hour, day of week, route, and station
- **Temporal**: Rolling mean delay (daily/weekly), trends by month
- **Missingness plot** and **correlation heatmap**

---

# 5) Feature Engineering & Metrics

## 5.1 Feature Groups
- **Time features**: hour, day_of_week, month + cyclical encoding (sin/cos)
- **Lag features**: `PREV_DELAY` per train/route/station
- **Rolling features**: 7-day rolling mean/std by route/train; event counts in last 7 days
- **Interaction features**: route × peak_hour, station × weather

## 5.2 Metrics
- **Regression**: RMSE, MAE, R²; MAE@TopDelay (for extreme delays)
- **Classification**: PR-AUC, F₂, recall, precision; threshold tuning

---

# 6) Modeling & Evaluation

- **Baselines**: Median predictor, Linear Regression, Ridge
- **Tree Ensembles**: RandomForest, XGBoost, LightGBM (CatBoost optional)
- **Cross-Validation**: TimeSeriesSplit
- **Hyperparameter Tuning**: Optuna (optimize on time-split folds)
- **Handling Imbalance** (classification): class_weight, threshold adjustment (cautious with SMOTE to avoid time leakage)

---

# 7) Comparison, Explainability & Deployment

- **Model Comparison**: Evaluate by primary metric, runtime, and stability across folds
- **Explainability**: Feature importance analysis + SHAP values (key drivers: peak_hour, route, rolling reliability metrics)
- **Deployment**: Save complete `pipeline = preprocessor + model` for inference; implement monitoring for data and performance drift

---

*Note: Update the data dictionary (metadata table) after loading and inspecting the actual dataset to ensure column names and thresholds align with business requirements.*

# Cell 1: Import Core Libraries

Import essential Python libraries including os for file operations, sys for system-specific parameters, numpy for numerical computations, and pandas for data manipulation.

In [1]:
import os, sys, numpy as np, pandas as pd

# Cell 2: Import Scikit-Learn Modules

Import key scikit-learn components: pipeline for building ML pipelines, metrics for evaluation, and preprocessing for data transformation.

In [2]:
from sklearn import pipeline, metrics, preprocessing

# Cell 3: Import Machine Learning Models

Import popular regression models: RandomForestRegressor, XGBRegressor, LGBMRegressor, and CatBoostRegressor for ensemble learning.

In [3]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
# CatBoost may not be installed in all environments — import safely
try:
    from catboost import CatBoostRegressor
    HAS_CATBOOST = True
except ModuleNotFoundError:
    CatBoostRegressor = None
    HAS_CATBOOST = False
    print("Warning: catboost not installed. Install with 'pip install catboost' to enable it.")

# Cell 4: Import Additional Libraries

Import optuna for hyperparameter optimization, joblib for model serialization, matplotlib and seaborn for data visualization.

In [4]:
import optuna, joblib
import matplotlib.pyplot as plt
import seaborn as sns

C:\Users\Admin\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Cell 5: Set Global Random Seed

Define a global random state for reproducibility across all random operations in the notebook.

In [5]:
RANDOM_STATE = 42

# Cell 6: Define Data Paths

Set up file paths for the training and testing datasets located in the docs directory.

In [6]:
DATA_PATH = r"D:\AnDB\L\mse\railway-delay\docs"
DATATRAIN = os.path.join(DATA_PATH, "merged_train_data.csv")
DATATEST = os.path.join(DATA_PATH, "railway-delay-dataset.csv")

# Cell 7: Define Column Constants

Specify the target column for prediction (delay_minutes) and the date column for time-based features.

In [7]:
TARGET_COL = 'DELAY_ARRIVAL'
DATE_COL = 'SCHEDULED_DEPARTURE'

# Cell 8: Memory Optimization Flags

Configure flags for handling large datasets: MAX_ROWS for limiting data size and DOWNSAMPLE for reducing sample size.

In [8]:
MAX_ROWS = None
DOWNSAMPLE = False

# Cell 9: Create Model Directory

Create a directory to store trained models and ensure it exists.

In [9]:
MODEL_DIR = "models"
os.makedirs(MODEL_DIR, exist_ok=True)

# Cell 10: Configuration Sanity Check

Verify that all paths are correct, files exist, and print configuration summary.

In [10]:
print("Data paths:")
print(f"Train data: {DATATRAIN}")
print(f"Test data: {DATATEST}")
print(f"Files exist: Train - {os.path.exists(DATATRAIN)}, Test - {os.path.exists(DATATEST)}")
print(f"Random state: {RANDOM_STATE}")
print(f"Target column: {TARGET_COL}")
print(f"Date column: {DATE_COL}")

Data paths:
Train data: D:\AnDB\L\mse\railway-delay\docs\merged_train_data.csv
Test data: D:\AnDB\L\mse\railway-delay\docs\railway-delay-dataset.csv
Files exist: Train - True, Test - True
Random state: 42
Target column: DELAY_ARRIVAL
Date column: SCHEDULED_DEPARTURE


# Group B — Load Data (Cells 11–20)

This section loads training and test datasets, detects and parses datetime columns, extracts time-based features (hour, weekday), and performs basic checks and sorting.

In [11]:
# Cell 11 – Read CSV (train)
print('\n--- Cell 11: Read train CSV ---')
try:
    df_train = pd.read_csv(DATATRAIN, nrows=MAX_ROWS, low_memory=False)
    print('Loaded train:', df_train.shape)
except Exception as e:
    print('Error reading train file:', e)
    df_train = None

if df_train is None:
    raise FileNotFoundError(f"Train file not found or could not be read: {DATATRAIN}")

if DOWNSAMPLE:
    print('Downsampling: using 10% sample of train')
    df_train = df_train.sample(frac=0.1, random_state=RANDOM_STATE).reset_index(drop=True)

TRAIN_DF = df_train
print('Cell 11 completed.')


--- Cell 11: Read train CSV ---
Loaded train: (9312671, 31)
Cell 11 completed.


# Cell 12: Read Test CSV (if available)

Read the test dataset if it exists; otherwise continue with train-only workflow.

In [12]:
# Cell 12 – Read CSV (test, nếu có)
print('\n--- Cell 12: Read test CSV (if exists) ---')
if os.path.exists(DATATEST):
    try:
        df_test = pd.read_csv(DATATEST, nrows=MAX_ROWS, low_memory=False)
        print('Loaded test:', df_test.shape)
    except Exception as e:
        print('Error reading test file:', e)
        df_test = None
else:
    df_test = None
    print('Test file not found; proceeding without test set')

TEST_DF = df_test
print('Cell 12 completed.')


--- Cell 12: Read test CSV (if exists) ---
Loaded test: (5819078, 31)
Cell 12 completed.


# Cell 13: Print columns & shapes

Quick check of columns and shapes for train and test dataframes.

In [13]:
# Cell 13 – Print columns
print('\n--- Cell 13: Train columns & shape ---')
print('Train shape:', TRAIN_DF.shape)
print('Train columns:', TRAIN_DF.columns.tolist())
if TEST_DF is not None:
    print('\n--- Test columns & shape ---')
    print('Test shape:', TEST_DF.shape)
    print('Test columns:', TEST_DF.columns.tolist())
print('Cell 13 completed.')


--- Cell 13: Train columns & shape ---
Train shape: (9312671, 31)
Train columns: ['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'TRAIN_OPERATOR', 'TRAIN_NUMBER', 'COACH_ID', 'SOURCE_STATION', 'DESTINATION_STATION', 'SCHEDULED_DEPARTURE', 'ACTUAL_DEPARTURE', 'DELAY_DEPARTURE', 'PLATFORM_TIME_OUT', 'TRAIN_DEPARTURE_EVENT', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'RUN_TIME', 'DISTANCE_KM', 'LEFT_SOURCE_STATION_TIME', 'PLATFORM_TIME_IN', 'SCHEDULED_ARRIVAL', 'ACTUAL_ARRIVAL', 'DELAY_ARRIVAL', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON', 'SYSTEM_DELAY', 'SECURITY_DELAY', 'TRAIN_OPERATOR_DELAY', 'LATE_TRAIN_DELAY', 'WEATHER_DELAY']

--- Test columns & shape ---
Test shape: (5819078, 31)
Test columns: ['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'TRAIN_OPERATOR', 'TRAIN_NUMBER', 'COACH_ID', 'SOURCE_STATION', 'DESTINATION_STATION', 'SCHEDULED_DEPARTURE', 'ACTUAL_DEPARTURE', 'DELAY_DEPARTURE', 'PLATFORM_TIME_OUT', 'TRAIN_DEPARTURE_EVENT', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'RUN_TIME', 'DISTANCE_KM', 'LEFT_SOU

# Cell 14: Auto-detect datetime column

Define a small helper to auto-detect datetime-like columns and choose the preferred one.

In [14]:
# Cell 14 – Auto-detect datetime column
import re

def detect_datetime_col(df, preferred=None):
    if df is None:
        return None
    if preferred and preferred in df.columns:
        return preferred
    candidates = [c for c in df.columns if re.search(r'date|time|scheduled|departure|arrival', c, flags=re.I)]
    if not candidates:
        return None
    for p in ['scheduled_departure','scheduled_time','scheduled_arrival','scheduled']:
        for c in candidates:
            if p in c.lower():
                return c
    return candidates[0]

TRAIN_DT_COL = detect_datetime_col(TRAIN_DF, preferred=DATE_COL)
TEST_DT_COL = detect_datetime_col(TEST_DF, preferred=DATE_COL) if TEST_DF is not None else None
print('Detected datetime columns -> train:', TRAIN_DT_COL, ', test:', TEST_DT_COL)
print('Cell 14 completed.')

Detected datetime columns -> train: SCHEDULED_DEPARTURE , test: SCHEDULED_DEPARTURE
Cell 14 completed.


# Cell 15: Parse datetime with pd.to_datetime

Coerce the detected datetime column(s) using pandas' to_datetime with error coercion.

In [15]:
# Cell 15 – pd.to_datetime
if TRAIN_DT_COL is not None:
    TRAIN_DF[TRAIN_DT_COL] = pd.to_datetime(TRAIN_DF[TRAIN_DT_COL].astype(str), errors='coerce', infer_datetime_format=True)
    print('Parsed train datetime: ', TRAIN_DF[TRAIN_DT_COL].notna().sum(), 'non-missing')
else:
    print('No datetime detected for train; skipping parse')

if TEST_DF is not None and TEST_DT_COL is not None:
    TEST_DF[TEST_DT_COL] = pd.to_datetime(TEST_DF[TEST_DT_COL].astype(str), errors='coerce', infer_datetime_format=True)
    print('Parsed test datetime: ', TEST_DF[TEST_DT_COL].notna().sum(), 'non-missing')

print('Cell 15 completed.')

C:\Users\Admin\AppData\Local\Temp\ipykernel_20728\1253054865.py:3: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  TRAIN_DF[TRAIN_DT_COL] = pd.to_datetime(TRAIN_DF[TRAIN_DT_COL].astype(str), errors='coerce', infer_datetime_format=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_20728\1253054865.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  TRAIN_DF[TRAIN_DT_COL] = pd.to_datetime(TRAIN_DF[TRAIN_DT_COL].astype(str), errors='coerce', infer_datetime_format=True)


Parsed train datetime:  2592697 non-missing


C:\Users\Admin\AppData\Local\Temp\ipykernel_20728\1253054865.py:9: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  TEST_DF[TEST_DT_COL] = pd.to_datetime(TEST_DF[TEST_DT_COL].astype(str), errors='coerce', infer_datetime_format=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_20728\1253054865.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  TEST_DF[TEST_DT_COL] = pd.to_datetime(TEST_DF[TEST_DT_COL].astype(str), errors='coerce', infer_datetime_format=True)


Parsed test datetime:  1620530 non-missing
Cell 15 completed.


# Cell 16: Extract hour and weekday

Create HOUR and WEEKDAY features from parsed datetime column(s).

In [16]:
# Cell 16 – Extract hour / weekday
if TRAIN_DT_COL is not None:
    TRAIN_DF[f'{TRAIN_DT_COL}_HOUR'] = TRAIN_DF[TRAIN_DT_COL].dt.hour
    TRAIN_DF[f'{TRAIN_DT_COL}_WEEKDAY'] = TRAIN_DF[TRAIN_DT_COL].dt.dayofweek
    print('Added train hour/weekday columns')
else:
    print('Train datetime not available; skipping hour/weekday extraction')

if TEST_DF is not None and TEST_DT_COL is not None:
    TEST_DF[f'{TEST_DT_COL}_HOUR'] = TEST_DF[TEST_DT_COL].dt.hour
    TEST_DF[f'{TEST_DT_COL}_WEEKDAY'] = TEST_DF[TEST_DT_COL].dt.dayofweek
    print('Added test hour/weekday columns')

print('Cell 16 completed.')

Added train hour/weekday columns
Added test hour/weekday columns
Cell 16 completed.


# Cell 17: Handle missing datetime

Attempt to fill missing datetime values from YEAR/MONTH/DAY + scheduled time columns (if available).

In [17]:
# Cell 17 – Handle missing datetime (fill from YEAR/MONTH/DAY + time)

def try_fill_datetime_from_parts(df, dt_col, time_cols_order=None):
    if df is None or dt_col is None:
        return df
    if not {'YEAR','MONTH','DAY'}.issubset(set(df.columns)):
        return df
    if time_cols_order is None:
        time_cols_order = ['SCHEDULED_DEPARTURE','SCHEDULED_TIME','SCHEDULED_ARRIVAL','ACTUAL_DEPARTURE','ACTUAL_ARRIVAL']
    time_cols = [c for c in time_cols_order if c in df.columns]
    if not time_cols:
        return df
    time_col = time_cols[0]

    def build_dt(row):
        try:
            y = int(row['YEAR'])
            m = int(row['MONTH'])
            d = int(row['DAY'])
            t = row[time_col]
            if pd.isna(t):
                return pd.NaT
            t_str = str(int(float(t)))
            if len(t_str) <= 2:
                hour = int(t_str)
                minute = 0
            else:
                if len(t_str) in (3,4):
                    hour = int(t_str[:-2])
                    minute = int(t_str[-2:])
                else:
                    hour = int(t_str)
                    minute = 0
            return pd.Timestamp(year=y, month=m, day=d, hour=hour, minute=minute)
        except Exception:
            return pd.NaT

    mask_missing = df[dt_col].isna()
    if mask_missing.any():
        df.loc[mask_missing, dt_col] = df[mask_missing].apply(build_dt, axis=1)
    return df

# Apply to train/test
if TRAIN_DT_COL is not None:
    miss_before = TRAIN_DF[TRAIN_DT_COL].isna().sum()
    TRAIN_DF = try_fill_datetime_from_parts(TRAIN_DF, TRAIN_DT_COL)
    miss_after = TRAIN_DF[TRAIN_DT_COL].isna().sum()
    print(f'Train datetime missing before: {miss_before}, after: {miss_after}')

if TEST_DF is not None and TEST_DT_COL is not None:
    miss_before = TEST_DF[TEST_DT_COL].isna().sum()
    TEST_DF = try_fill_datetime_from_parts(TEST_DF, TEST_DT_COL)
    miss_after = TEST_DF[TEST_DT_COL].isna().sum()
    print(f'Test datetime missing before: {miss_before}, after: {miss_after}')

print('Cell 17 completed.')

Train datetime missing before: 6719974, after: 6719974
Test datetime missing before: 4198548, after: 4198548
Cell 17 completed.


# Cell 18: Sort by time

Sort the dataframe by the parsed datetime column and reset the index.

In [18]:
# Cell 18 – Sort by time
if TRAIN_DT_COL is not None:
    TRAIN_DF = TRAIN_DF.sort_values(by=TRAIN_DT_COL).reset_index(drop=True)
    print('Train sorted by', TRAIN_DT_COL)
else:
    print('Train datetime missing; skipping sort')

if TEST_DF is not None and TEST_DT_COL is not None:
    TEST_DF = TEST_DF.sort_values(by=TEST_DT_COL).reset_index(drop=True)
    print('Test sorted by', TEST_DT_COL)

print('Cell 18 completed.')

Train sorted by SCHEDULED_DEPARTURE
Test sorted by SCHEDULED_DEPARTURE
Cell 18 completed.


# Cell 19: Basic data info

Show dataframe info and top missing counts.

In [19]:
# Cell 19 – Basic data info (df.info())
print('\n--- Train info ---')
TRAIN_DF.info()
print('\nTop missing counts (train):')
print(TRAIN_DF.isna().sum().sort_values(ascending=False).head(15))

if TEST_DF is not None:
    print('\n--- Test info ---')
    TEST_DF.info()
    print('\nTop missing counts (test):')
    print(TEST_DF.isna().sum().sort_values(ascending=False).head(15))

print('Cell 19 completed.')


--- Train info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9312671 entries, 0 to 9312670
Data columns (total 33 columns):
 #   Column                       Dtype         
---  ------                       -----         
 0   YEAR                         object        
 1   MONTH                        object        
 2   DAY                          object        
 3   DAY_OF_WEEK                  object        
 4   TRAIN_OPERATOR               object        
 5   TRAIN_NUMBER                 object        
 6   COACH_ID                     object        
 7   SOURCE_STATION               object        
 8   DESTINATION_STATION          object        
 9   SCHEDULED_DEPARTURE          datetime64[ns]
 10  ACTUAL_DEPARTURE             object        
 11  DELAY_DEPARTURE              object        
 12  PLATFORM_TIME_OUT            object        
 13  TRAIN_DEPARTURE_EVENT        object        
 14  SCHEDULED_TIME               object        
 15  ELAPSED_TIME                 

# Cell 20: Head / sample view

Display the first rows and a small random sample to inspect data content.

In [20]:
# Cell 20 – Head / sample view
print('\n--- Train head ---')
print(TRAIN_DF.head())
print('\n--- Train random sample ---')
print(TRAIN_DF.sample(min(5, len(TRAIN_DF)), random_state=RANDOM_STATE))

if TEST_DF is not None:
    print('\n--- Test head ---')
    print(TEST_DF.head())

print('Cell 20 completed. Group B finished.')


--- Train head ---
   YEAR MONTH DAY DAY_OF_WEEK TRAIN_OPERATOR TRAIN_NUMBER COACH_ID  \
0  2015     1   1           4             UA          144   N66056   
1  2015     4  13           1             AA          247   N5EDAA   
2  2015     4  13           1             AA          243   N476AA   
3  2015     4  13           1             AA          199   N3DHAA   
4  2015     4  13           1             AA           45   N3CPAA   

  SOURCE_STATION DESTINATION_STATION SCHEDULED_DEPARTURE  ... DIVERTED  \
0            HNL                 IAD          1700-01-01  ...        0   
1            LAX                 KOA          1700-01-01  ...        0   
2            DFW                 SAT          1700-01-01  ...        0   
3            JFK                 ORD          1700-01-01  ...        0   
4            JFK                 LAS          1700-01-01  ...        0   

  CANCELLED CANCELLATION_REASON SYSTEM_DELAY SECURITY_DELAY  \
0         0                 NaN          NaN       

In [21]:
# Cells 21–29: Helper functions for feature engineering, metrics, plotting, and inference
import typing as t
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import precision_recall_curve, average_precision_score, roc_curve, auc, confusion_matrix, fbeta_score, precision_score, recall_score


# Cell 21 – _get_route_column
def _get_route_column(df: pd.DataFrame, src_col: str = 'SOURCE_STATION', dst_col: str = 'DESTINATION_STATION', train_col: str = 'TRAIN_NUMBER', route_col: str = 'ROUTE') -> str:
    """Create or return a route identifier column name.

    Preference: if both source and destination exist, build 'SRC_DST'. Else fall back to TRAIN_NUMBER.
    Returns the route column name (added if not present).
    """
    if src_col in df.columns and dst_col in df.columns:
        if route_col not in df.columns:
            df[route_col] = df[src_col].astype(str) + '_' + df[dst_col].astype(str)
        return route_col
    if train_col in df.columns:
        return train_col
    raise ValueError('Neither route nor train identifier columns found')


# Cell 22 – compute_prev_delay_safe
def compute_prev_delay_safe(df: pd.DataFrame, groupby_cols: t.List[str], dt_col: str, delay_col: str, out_col: str = 'PREV_DELAY') -> pd.DataFrame:
    """Compute previous delay (lag-1) per group in a time-safe manner.

    Assumes dt_col is parsed datetime and df is sorted by dt_col.
    """
    df = df.copy()
    if dt_col not in df.columns or delay_col not in df.columns:
        df[out_col] = np.nan
        return df
    df = df.sort_values(by=dt_col)
    df[out_col] = df.groupby(groupby_cols)[delay_col].shift(1)
    # option: fill missing with 0 or keep NaN — keep NaN to avoid leakage
    return df


# Cell 23 – compute_rolling_features_safe
def compute_rolling_features_safe(df: pd.DataFrame, groupby_col: str, dt_col: str, value_col: str, window: str = '7D', min_periods: int = 1) -> pd.DataFrame:
    """Compute rolling mean/std/count over a time window per group (time-based rolling).

    Returns new columns: ROLL_MEAN_{window}, ROLL_STD_{window}, ROLL_COUNT_{window}
    """
    df = df.copy()
    if dt_col not in df.columns or value_col not in df.columns:
        return df
    # ensure datetime index
    df = df.sort_values(by=dt_col).reset_index(drop=True)
    col_mean = f'ROLL_MEAN_{window}'
    col_std = f'ROLL_STD_{window}'
    col_count = f'ROLL_COUNT_{window}'

    # set index for rolling
    tmp = df.set_index(pd.DatetimeIndex(df[dt_col]))
    rolled = tmp.groupby(groupby_col)[value_col].rolling(window=window, min_periods=min_periods).agg(['mean','std','count']).reset_index(level=0, drop=True)
    df[col_mean] = rolled['mean'].values
    df[col_std] = rolled['std'].values
    df[col_count] = rolled['count'].values
    return df


# Cell 24 – metrics_summary
def metrics_summary_regression(y_true: t.Iterable, y_pred: t.Iterable) -> dict:
    return {
        'rmse': mean_squared_error(y_true, y_pred, squared=False),
        'mae': mean_absolute_error(y_true, y_pred),
        'r2': r2_score(y_true, y_pred)
    }


def metrics_summary_classification(y_true: t.Iterable, y_proba: t.Iterable, threshold: float = 0.5, beta: float = 2.0) -> dict:
    y_pred = (np.array(y_proba) >= threshold).astype(int)
    ap = average_precision_score(y_true, y_proba)
    fbeta = fbeta_score(y_true, y_pred, beta=beta)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    return {'ap': ap, 'fbeta': fbeta, 'precision': prec, 'recall': rec, 'threshold': threshold}


# Cell 25 – plotting helpers (PR / ROC)
def plot_pr_roc(y_true: t.Iterable, y_score: t.Iterable, figsize=(8,4)):
    plt.figure(figsize=(14,4))
    plt.subplot(1,2,1)
    precision, recall, _ = precision_recall_curve(y_true, y_score)
    ap = average_precision_score(y_true, y_score)
    plt.plot(recall, precision, label=f'AP={ap:.3f}')
    plt.xlabel('Recall'); plt.ylabel('Precision'); plt.title('Precision-Recall'); plt.legend()

    plt.subplot(1,2,2)
    fpr, tpr, _ = roc_curve(y_true, y_score)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'AUC={roc_auc:.3f}')
    plt.plot([0,1],[0,1],'k--', alpha=0.3)
    plt.xlabel('FPR'); plt.ylabel('TPR'); plt.title('ROC'); plt.legend()
    plt.tight_layout()


# Cell 26 – confusion matrix helper
def plot_confusion(y_true: t.Iterable, y_pred: t.Iterable, labels: t.List = None):
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Pred'); plt.ylabel('True'); plt.title('Confusion Matrix')


# Cell 27 – threshold tuning
def threshold_tuning(y_true: t.Iterable, y_score: t.Iterable, beta: float = 2.0):
    thresholds = np.linspace(0,1,101)
    results = []
    y_true = np.array(y_true)
    y_score = np.array(y_score)
    for t in thresholds:
        y_pred = (y_score >= t).astype(int)
        f2 = fbeta_score(y_true, y_pred, beta=beta)
        ap = average_precision_score(y_true, y_score)
        prec = precision_score(y_true, y_pred, zero_division=0)
        rec = recall_score(y_true, y_pred, zero_division=0)
        results.append({'threshold': t, 'fbeta': f2, 'ap': ap, 'precision': prec, 'recall': rec})
    res_df = pd.DataFrame(results)
    best = res_df.loc[res_df['fbeta'].idxmax()]
    return res_df, best.to_dict()


# Cell 28 – inference helper
def run_inference(pipeline, X: pd.DataFrame, threshold: float = None):
    """Run pipeline.predict / predict_proba and return predictions and optionally labels using threshold.
    Pipeline may be a sklearn pipeline or a trained model with predict_proba.
    """
    preds = pipeline.predict(X)
    proba = None
    if hasattr(pipeline, 'predict_proba'):
        try:
            proba = pipeline.predict_proba(X)
            # if binary, proba[:,1]
            if proba.ndim == 2 and proba.shape[1] > 1:
                proba = proba[:,1]
        except Exception:
            proba = None
    if threshold is not None and proba is not None:
        labels = (np.array(proba) >= threshold).astype(int)
    elif threshold is not None:
        labels = (np.array(preds) >= threshold).astype(int)
    else:
        labels = None
    return {'pred': preds, 'proba': proba, 'label': labels}


# Cell 29 – helper sanity tests
def _sanity_tests():
    print('Running simple sanity tests...')
    import sklearn
    # small synthetic dataset
    df = pd.DataFrame({'YEAR':[2020,2020],'MONTH':[1,1],'DAY':[1,1],'SCHEDULED_DEPARTURE':[8,9],'DELAY_ARRIVAL':[2,5], 'SOURCE_STATION':['A','A'],'DESTINATION_STATION':['B','B'],'TRAIN_NUMBER':[1,1]})
    rcol = _get_route_column(df)
    df = compute_prev_delay_safe(df, groupby_cols=[rcol], dt_col=None, delay_col='DELAY_ARRIVAL')
    print('Sanity tests passed (basic run)')

# expose for import
_helpers = {'get_route': _get_route_column, 'prev_delay': compute_prev_delay_safe, 'rolling': compute_rolling_features_safe, 'metrics_reg': metrics_summary_regression, 'metrics_clf': metrics_summary_classification}
print('Group C helpers (Cells 21–29) added.')

Group C helpers (Cells 21–29) added.


# Cell 30: End helper section

End of Group C — helper functions. Proceed to Feature Engineering (Group D) when ready.

# Group D — Feature Engineering (Cells 31–45)

This group creates time features, cyclical encodings, lag and rolling features (route-based and global fallback), fills missing engineered features, performs basic checks, drops obvious leakage columns, and optionally saves an intermediate dataset.

In [22]:
# Cell 31 – Time features (HOUR, DAY_OF_WEEK, MONTH)
print('\n--- Cell 31: Time features ---')
if TRAIN_DT_COL is not None:
    TRAIN_DF['HOUR'] = TRAIN_DF[TRAIN_DT_COL].dt.hour
    TRAIN_DF['DAY_OF_WEEK'] = TRAIN_DF[TRAIN_DT_COL].dt.dayofweek
    TRAIN_DF['MONTH'] = TRAIN_DF[TRAIN_DT_COL].dt.month
    print('Added HOUR, DAY_OF_WEEK, MONTH to TRAIN_DF')
else:
    print('TRAIN_DT_COL not set; cannot derive time features for train')

if TEST_DF is not None and TEST_DT_COL is not None:
    TEST_DF['HOUR'] = TEST_DF[TEST_DT_COL].dt.hour
    TEST_DF['DAY_OF_WEEK'] = TEST_DF[TEST_DT_COL].dt.dayofweek
    TEST_DF['MONTH'] = TEST_DF[TEST_DT_COL].dt.month
    print('Added HOUR, DAY_OF_WEEK, MONTH to TEST_DF')

print('Cell 31 completed.')


--- Cell 31: Time features ---
Added HOUR, DAY_OF_WEEK, MONTH to TRAIN_DF
Added HOUR, DAY_OF_WEEK, MONTH to TEST_DF
Cell 31 completed.


# Cell 32: Cyclical encoding (SIN, COS)

Encode hour and weekday as cyclical features to preserve circularity.

In [23]:
# Cell 32 – Cyclical encoding (SIN, COS)
print('\n--- Cell 32: Cyclical encoding ---')
if 'HOUR' in TRAIN_DF.columns:
    TRAIN_DF['HOUR_SIN'] = np.sin(2 * np.pi * TRAIN_DF['HOUR'] / 24)
    TRAIN_DF['HOUR_COS'] = np.cos(2 * np.pi * TRAIN_DF['HOUR'] / 24)
    print('Added HOUR_SIN, HOUR_COS')
if 'DAY_OF_WEEK' in TRAIN_DF.columns:
    TRAIN_DF['DOW_SIN'] = np.sin(2 * np.pi * TRAIN_DF['DAY_OF_WEEK'] / 7)
    TRAIN_DF['DOW_COS'] = np.cos(2 * np.pi * TRAIN_DF['DAY_OF_WEEK'] / 7)
    print('Added DOW_SIN, DOW_COS')

if TEST_DF is not None:
    if 'HOUR' in TEST_DF.columns:
        TEST_DF['HOUR_SIN'] = np.sin(2 * np.pi * TEST_DF['HOUR'] / 24)
        TEST_DF['HOUR_COS'] = np.cos(2 * np.pi * TEST_DF['HOUR'] / 24)
    if 'DAY_OF_WEEK' in TEST_DF.columns:
        TEST_DF['DOW_SIN'] = np.sin(2 * np.pi * TEST_DF['DAY_OF_WEEK'] / 7)
        TEST_DF['DOW_COS'] = np.cos(2 * np.pi * TEST_DF['DAY_OF_WEEK'] / 7)

print('Cell 32 completed.')


--- Cell 32: Cyclical encoding ---
Added HOUR_SIN, HOUR_COS
Added DOW_SIN, DOW_COS
Cell 32 completed.


# Cell 33: Lag feature (PREV_DELAY)

Compute previous delay per route/train to capture chain-effect.

In [24]:
# Cell 33 – Lag feature (PREV_DELAY)
print('\n--- Cell 33: Compute PREV_DELAY (per route) ---')
try:
    route_col = _get_route_column(TRAIN_DF)
    TRAIN_DF = compute_prev_delay_safe(TRAIN_DF, groupby_cols=[route_col], dt_col=TRAIN_DT_COL, delay_col=TARGET_COL, out_col='PREV_DELAY')
    print('Added PREV_DELAY per', route_col)
    if TEST_DF is not None:
        try:
            # for test, compute prev delay using train+test concatenation by route to avoid leakage issues in this quick step
            combined = pd.concat([TRAIN_DF, TEST_DF], sort=False, ignore_index=True)
            combined = compute_prev_delay_safe(combined, groupby_cols=[route_col], dt_col=TRAIN_DT_COL if TRAIN_DT_COL in combined.columns else TEST_DT_COL, delay_col=TARGET_COL, out_col='PREV_DELAY')
            # copy back PREV_DELAY for test rows
            TEST_DF['PREV_DELAY'] = combined.loc[combined.index >= len(TRAIN_DF), 'PREV_DELAY'].values
        except Exception:
            TEST_DF['PREV_DELAY'] = np.nan
except Exception as e:
    print('Could not compute PREV_DELAY:', e)
    TRAIN_DF['PREV_DELAY'] = np.nan

print('Cell 33 completed.')


--- Cell 33: Compute PREV_DELAY (per route) ---
Added PREV_DELAY per ROUTE
Cell 33 completed.


# Cell 34: Rolling mean 7D (route-based)

Compute rolling mean/std/count for the target over a 7-day window per route.

In [25]:
# Cell 34 – Rolling mean 7D (route-based)
print('\n--- Cell 34: Rolling features (7D) per route ---')
try:
    TRAIN_DF = compute_rolling_features_safe(TRAIN_DF, groupby_col=route_col, dt_col=TRAIN_DT_COL, value_col=TARGET_COL, window='7D', min_periods=1)
    print('Added rolling features (route-based)')
    if TEST_DF is not None:
        # For simplicity in this quick pipeline, compute rolling on combined set and slice back
        combined = pd.concat([TRAIN_DF, TEST_DF], sort=False, ignore_index=True)
        combined = compute_rolling_features_safe(combined, groupby_col=route_col, dt_col=TRAIN_DT_COL if TRAIN_DT_COL in combined.columns else TEST_DT_COL, value_col=TARGET_COL, window='7D', min_periods=1)
        TEST_DF['ROLL_MEAN_7D'] = combined.loc[combined.index >= len(TRAIN_DF), 'ROLL_MEAN_7D'].values
        TEST_DF['ROLL_STD_7D'] = combined.loc[combined.index >= len(TRAIN_DF), 'ROLL_STD_7D'].values
        TEST_DF['ROLL_COUNT_7D'] = combined.loc[combined.index >= len(TRAIN_DF), 'ROLL_COUNT_7D'].values
except Exception as e:
    print('Could not compute rolling features:', e)

print('Cell 34 completed.')


--- Cell 34: Rolling features (7D) per route ---
Could not compute rolling features: index values must not have NaT
Cell 34 completed.


# Cell 35: Rolling mean global fallback

Compute a global rolling mean so route-level NaNs can fall back to a global trend.

In [26]:
# Cell 35 – Rolling mean global fallback
print('\n--- Cell 35: Global rolling mean fallback (7D) ---')
try:
    TRAIN_DF = TRAIN_DF.set_index(TRAIN_DT_COL)
    TRAIN_DF['GLOBAL_ROLL_MEAN_7D'] = TRAIN_DF[TARGET_COL].rolling('7D', min_periods=1).mean()
    TRAIN_DF = TRAIN_DF.reset_index()
    # fill route-level ROLL_MEAN_7D with global when missing
    if 'ROLL_MEAN_7D' in TRAIN_DF.columns:
        TRAIN_DF['ROLL_MEAN_7D'] = TRAIN_DF['ROLL_MEAN_7D'].fillna(TRAIN_DF['GLOBAL_ROLL_MEAN_7D'])
    print('Computed GLOBAL_ROLL_MEAN_7D and applied fallback')
    if TEST_DF is not None:
        try:
            combined = pd.concat([TRAIN_DF, TEST_DF], sort=False, ignore_index=True).set_index(TRAIN_DT_COL)
            combined['GLOBAL_ROLL_MEAN_7D'] = combined[TARGET_COL].rolling('7D', min_periods=1).mean()
            combined = combined.reset_index()
            TEST_DF['GLOBAL_ROLL_MEAN_7D'] = combined.loc[combined.index >= len(TRAIN_DF), 'GLOBAL_ROLL_MEAN_7D'].values
            if 'ROLL_MEAN_7D' in TEST_DF.columns:
                TEST_DF['ROLL_MEAN_7D'] = TEST_DF['ROLL_MEAN_7D'].fillna(TEST_DF['GLOBAL_ROLL_MEAN_7D'])
        except Exception:
            TEST_DF['GLOBAL_ROLL_MEAN_7D'] = np.nan
except Exception as e:
    print('Could not compute GLOBAL_ROLL_MEAN_7D:', e)

print('Cell 35 completed.')


--- Cell 35: Global rolling mean fallback (7D) ---
Could not compute GLOBAL_ROLL_MEAN_7D: index values must not have NaT
Cell 35 completed.


# Cell 36: Weather / external features (if available)

Check for weather / external columns and add placeholder features or flags.

In [27]:
# Cell 36 – Weather / external features (nếu có)
print('\n--- Cell 36: External / weather features check ---')
weather_cols = [c for c in TRAIN_DF.columns if 'WEATHER' in c.upper() or 'TEMPERATURE' in c.upper() or 'PRECIP' in c.upper()]
if weather_cols:
    print('Found weather columns:', weather_cols)
    # example: create a simple severity flag if 'WEATHER_SEVERITY' exists
    if 'WEATHER_SEVERITY' in TRAIN_DF.columns:
        TRAIN_DF['WEATHER_SEV_FLAG'] = (TRAIN_DF['WEATHER_SEVERITY'] > 1).astype(int)
        print('Added WEATHER_SEV_FLAG')
else:
    print('No weather/external columns detected — skipping')

print('Cell 36 completed.')


--- Cell 36: External / weather features check ---
Found weather columns: ['WEATHER_DELAY']
Cell 36 completed.


# Cell 37: Fill missing engineered features

Fill NaNs for PREV_DELAY and rolling features with sensible defaults to avoid downstream errors.

In [28]:
# Cell 37 – Fill missing engineered features
print('\n--- Cell 37: Filling missing engineered features ---')
# PREV_DELAY: fill with 0 (assume no info => treat as 0 previous delay)
if 'PREV_DELAY' in TRAIN_DF.columns:
    TRAIN_DF['PREV_DELAY'] = TRAIN_DF['PREV_DELAY'].fillna(0)
if TEST_DF is not None and 'PREV_DELAY' in TEST_DF.columns:
    TEST_DF['PREV_DELAY'] = TEST_DF['PREV_DELAY'].fillna(0)

# Rolling mean: fill with GLOBAL_ROLL_MEAN_7D or median
for col in ['ROLL_MEAN_7D','ROLL_STD_7D','ROLL_COUNT_7D','GLOBAL_ROLL_MEAN_7D']:
    if col in TRAIN_DF.columns:
        TRAIN_DF[col] = TRAIN_DF[col].fillna(TRAIN_DF[col].median())
    if TEST_DF is not None and col in TEST_DF.columns:
        TEST_DF[col] = TEST_DF[col].fillna(TRAIN_DF[col].median() if col in TRAIN_DF.columns else 0)

# Cyclical NaNs: replace with -1 which indicates missing
for col in ['HOUR_SIN','HOUR_COS','DOW_SIN','DOW_COS']:
    if col in TRAIN_DF.columns:
        TRAIN_DF[col] = TRAIN_DF[col].fillna(-1)
    if TEST_DF is not None and col in TEST_DF.columns:
        TEST_DF[col] = TEST_DF[col].fillna(-1)

print('Cell 37 completed.')


--- Cell 37: Filling missing engineered features ---


C:\Users\Admin\AppData\Local\Temp\ipykernel_20728\3932380468.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  TEST_DF['PREV_DELAY'] = TEST_DF['PREV_DELAY'].fillna(0)


Cell 37 completed.


# Cell 38: Feature distribution check

Inspect distributions of the engineered features (describe + optional histograms).

In [29]:
# Cell 38 – Feature distribution check
print('\n--- Cell 38: Feature distribution check ---')
eng_cols = [c for c in TRAIN_DF.columns if any(pat in c for pat in ['PREV_DELAY','ROLL_','HOUR','DOW','GLOBAL_ROLL'])]
print('Engineered columns count:', len(eng_cols))
print(TRAIN_DF[eng_cols].describe().T)

# Optional quick plots (commented out by default)
# for c in eng_cols:
#     plt.figure(); sns.histplot(TRAIN_DF[c].dropna(), bins=50); plt.title(c)

print('Cell 38 completed.')


--- Cell 38: Feature distribution check ---
Engineered columns count: 7
                              count      mean       std  min  25%  50%  \
SCHEDULED_DEPARTURE_HOUR  2592697.0  0.000000  0.000000  0.0  0.0  0.0   
HOUR                      2592697.0  0.000000  0.000000  0.0  0.0  0.0   
HOUR_SIN                  9312671.0 -0.721595  0.448214 -1.0 -1.0 -1.0   
HOUR_COS                  9312671.0 -0.443189  0.896428 -1.0 -1.0 -1.0   
DOW_SIN                   9312671.0 -0.722138  0.583850 -1.0 -1.0 -1.0   
DOW_COS                   9312671.0 -0.730736  0.570311 -1.0 -1.0 -1.0   

                               75%       max  
SCHEDULED_DEPARTURE_HOUR  0.000000  0.000000  
HOUR                      0.000000  0.000000  
HOUR_SIN                  0.000000  0.000000  
HOUR_COS                  1.000000  1.000000  
DOW_SIN                  -0.974928  0.974928  
DOW_COS                  -0.900969  1.000000  
Cell 38 completed.


# Cell 39: Drop leakage columns

Remove columns that clearly leak future information (e.g., ACTUAL_* columns).

In [30]:
# Cell 39 – Drop leakage columns
print('\n--- Cell 39: Drop leakage columns ---')
leak_cols = [c for c in TRAIN_DF.columns if c.upper().startswith('ACTUAL_') or c.upper().startswith('LEFT_')]
# do not drop target or scheduled columns
leak_cols = [c for c in leak_cols if c not in [TRAIN_DT_COL, TARGET_COL]]
print('Leakage candidate columns to drop:', leak_cols)
if leak_cols:
    TRAIN_DF = TRAIN_DF.drop(columns=leak_cols)
    if TEST_DF is not None:
        TEST_DF = TEST_DF.drop(columns=[c for c in leak_cols if c in TEST_DF.columns], errors='ignore')
    print('Dropped leakage columns')
else:
    print('No obvious leakage columns found')

print('Cell 39 completed.')


--- Cell 39: Drop leakage columns ---
Leakage candidate columns to drop: ['ACTUAL_DEPARTURE', 'LEFT_SOURCE_STATION_TIME', 'ACTUAL_ARRIVAL']
Dropped leakage columns
Cell 39 completed.


# Cell 40: Feature list snapshot

Take a snapshot of feature column names for downstream modeling.

In [31]:
# Cell 40 – Feature list snapshot
print('\n--- Cell 40: Feature list snapshot ---')
reserved = {TARGET_COL, TRAIN_DT_COL}
features = [c for c in TRAIN_DF.columns if c not in reserved]
print('Total columns:', len(TRAIN_DF.columns), 'Feature candidate count:', len(features))
print('First 50 features:', features[:50])
FEATURE_LIST = features  # expose for downstream
print('Cell 40 completed.')


--- Cell 40: Feature list snapshot ---
Total columns: 36 Feature candidate count: 35
First 50 features: ['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'TRAIN_OPERATOR', 'TRAIN_NUMBER', 'COACH_ID', 'SOURCE_STATION', 'DESTINATION_STATION', 'DELAY_DEPARTURE', 'PLATFORM_TIME_OUT', 'TRAIN_DEPARTURE_EVENT', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'RUN_TIME', 'DISTANCE_KM', 'PLATFORM_TIME_IN', 'SCHEDULED_ARRIVAL', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON', 'SYSTEM_DELAY', 'SECURITY_DELAY', 'TRAIN_OPERATOR_DELAY', 'LATE_TRAIN_DELAY', 'WEATHER_DELAY', 'SCHEDULED_DEPARTURE_HOUR', 'SCHEDULED_DEPARTURE_WEEKDAY', 'HOUR', 'HOUR_SIN', 'HOUR_COS', 'DOW_SIN', 'DOW_COS', 'ROUTE', 'PREV_DELAY']
Cell 40 completed.


# Cell 41: Feature sanity check

Verify missingness and types for the main engineered features.

In [32]:
# Cell 41 – Feature sanity check
print('\n--- Cell 41: Feature sanity check ---')
check_cols = ['PREV_DELAY','ROLL_MEAN_7D','ROLL_STD_7D','ROLL_COUNT_7D','GLOBAL_ROLL_MEAN_7D','HOUR','DAY_OF_WEEK']
present = [c for c in check_cols if c in TRAIN_DF.columns]
print('Present engineered columns:', present)
for c in present:
    pct_na = TRAIN_DF[c].isna().mean() * 100
    print(f"{c}: dtype={TRAIN_DF[c].dtype}, NA%={pct_na:.2f}")
print('Cell 41 completed.')


--- Cell 41: Feature sanity check ---
Present engineered columns: ['PREV_DELAY', 'HOUR', 'DAY_OF_WEEK']
PREV_DELAY: dtype=object, NA%=0.00
HOUR: dtype=float64, NA%=72.16
DAY_OF_WEEK: dtype=float64, NA%=72.16
Cell 41 completed.


# Cell 42: Print engineered columns

List columns created by feature engineering for quick inspection.

In [33]:
# Cell 42 – Print engineered columns
eng_cols = [c for c in TRAIN_DF.columns if any(pat in c for pat in ['PREV_DELAY','ROLL_','GLOBAL_ROLL','HOUR_','DOW_','HOUR','DAY_OF_WEEK'])]
print('\n--- Cell 42: Engineered columns ---')
print(sorted(set(eng_cols)))
print('Cell 42 completed.')


--- Cell 42: Engineered columns ---
['DAY_OF_WEEK', 'DOW_COS', 'DOW_SIN', 'HOUR', 'HOUR_COS', 'HOUR_SIN', 'PREV_DELAY', 'SCHEDULED_DEPARTURE_HOUR']
Cell 42 completed.


# Cell 43: Memory cleanup

Drop temporary variables and run garbage collection to free memory.

In [34]:
# Cell 43 – Memory cleanup
print('\n--- Cell 43: Memory cleanup ---')
import gc
# drop large temporaries if present
for name in ['combined','tmp']:
    if name in globals():
        del globals()[name]
gc.collect()
print('Memory cleanup done')
print('Cell 43 completed.')


--- Cell 43: Memory cleanup ---
Memory cleanup done
Cell 43 completed.


# Cell 44: Save intermediate df (optional)

Save processed train dataframe to a parquet file for faster iteration/use in downstream steps.

In [35]:
# Cell 44 – Save intermediate df (optional)
print('\n--- Cell 44: Save intermediate dataframe (optional) ---')
OUT_DIR = os.path.join('data','interim')
os.makedirs(OUT_DIR, exist_ok=True)
out_path = os.path.join(OUT_DIR, 'train_fe.parquet')
try:
    TRAIN_DF.to_parquet(out_path, index=False)
    print('Saved intermediate TRAIN_DF to', out_path)
except Exception as e:
    print('Could not save TRAIN_DF:', e)

print('Cell 44 completed.')


--- Cell 44: Save intermediate dataframe (optional) ---
Could not save TRAIN_DF: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.
Cell 44 completed.


# Cell 45: End feature engineering marker

End of Group D — Feature Engineering. Proceed to Group E (Target & Feature Split) when ready.

# Group E — Target & Feature Split (Cells 46–50)

Define features/target, optionally log-transform the target, save an original copy, and print basic target statistics.

In [36]:
# Cell 46 – Define X, y
print('\n--- Cell 46: Define X and y ---')
# Ensure FEATURE_LIST exists; if not, compute conservatively
if 'FEATURE_LIST' not in globals() or FEATURE_LIST is None:
    reserved = {TARGET_COL, TRAIN_DT_COL}
    FEATURE_LIST = [c for c in TRAIN_DF.columns if c not in reserved]

print('Using', len(FEATURE_LIST), 'features (sample):', FEATURE_LIST[:20])
X = TRAIN_DF[FEATURE_LIST].copy()
y = TRAIN_DF[TARGET_COL].copy()
X_train = X
y_train = y
print('Defined X_train (shape=', X_train.shape, ') and y_train (length=', len(y_train), ')')
print('Cell 46 completed.')


--- Cell 46: Define X and y ---
Using 35 features (sample): ['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'TRAIN_OPERATOR', 'TRAIN_NUMBER', 'COACH_ID', 'SOURCE_STATION', 'DESTINATION_STATION', 'DELAY_DEPARTURE', 'PLATFORM_TIME_OUT', 'TRAIN_DEPARTURE_EVENT', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'RUN_TIME', 'DISTANCE_KM', 'PLATFORM_TIME_IN', 'SCHEDULED_ARRIVAL', 'DIVERTED', 'CANCELLED']
Defined X_train (shape= (9312671, 35) ) and y_train (length= 9312671 )
Cell 46 completed.


In [37]:
# Cell 47 – Log-transform target (nếu có)
print('\n--- Cell 47: Optional log-transform of target ---')
# Toggle this to True to apply log1p transform to the target for modeling stability
LOG_TRANSFORM_TARGET = False
if LOG_TRANSFORM_TARGET:
    TRAIN_DF['TARGET_LOG'] = np.log1p(TRAIN_DF[TARGET_COL].clip(lower=0))
    y_train = TRAIN_DF['TARGET_LOG']
    print('Applied log1p transform to target; sample stats:')
    print(TRAIN_DF['TARGET_LOG'].describe())
else:
    print('LOG_TRANSFORM_TARGET is False — no transform applied')
print('Cell 47 completed.')


--- Cell 47: Optional log-transform of target ---
LOG_TRANSFORM_TARGET is False — no transform applied
Cell 47 completed.


In [38]:
# Cell 48 – Save original target
print('\n--- Cell 48: Save original target column ---')
if 'TARGET_ORIG' not in TRAIN_DF.columns:
    TRAIN_DF['TARGET_ORIG'] = TRAIN_DF[TARGET_COL].copy()
    print('Saved original target to TRAIN_DF["TARGET_ORIG"]')
else:
    print('TARGET_ORIG already exists; skipping')

print('Cell 48 completed.')


--- Cell 48: Save original target column ---
Saved original target to TRAIN_DF["TARGET_ORIG"]
Cell 48 completed.


In [ ]:
# Cell 49 – Target stats
print('\n--- Cell 49: Target statistics ---')
if TARGET_COL in TRAIN_DF.columns:
    raw_t = TRAIN_DF[TARGET_COL]
    # detect non-numeric entries before coercion
    non_numeric_mask = pd.to_numeric(raw_t, errors='coerce').isna() & raw_t.notna()
    n_non_numeric = int(non_numeric_mask.sum())
    if n_non_numeric > 0:
        print('Non-numeric target entries (count):', n_non_numeric)
        print('Sample non-numeric values:')
        print(raw_t[non_numeric_mask].unique()[:10])
    # coerce to numeric for statistics (invalid -> NaN -> dropped)
    t = pd.to_numeric(raw_t, errors='coerce').dropna()
    print('Count (numeric):', len(t))
    if len(t) > 0:
        print(t.describe())
        print('Skew:', t.skew())
        print('Negative values (count):', (t < 0).sum())
        print('Zeros (count):', (t == 0).sum())
        print('Quantiles (1%, 5%, 25%, 50%, 75%, 95%, 99%):', t.quantile([0.01,0.05,0.25,0.5,0.75,0.95,0.99]).to_dict())
    else:
        print('No numeric target values after coercion.')
else:
    print('TARGET_COL not found in TRAIN_DF')

print('Cell 49 completed.')

# Cell 50: End target section

End of Group E — Target & Feature Split. Proceed to Group F (Feature types & Preprocessing) when ready.

# Group F — Feature Types & Preprocessing (Cells 51–60)

Detect numeric/categorical/label features, check for leakage, and build numeric and categorical pipelines plus a ColumnTransformer preprocessor.

In [41]:
# Cell 51 – Detect numeric features
print('\n--- Cell 51: Detect numeric features ---')
NUMERIC_FEATURES = [c for c in FEATURE_LIST if pd.api.types.is_numeric_dtype(TRAIN_DF[c])]
print('Detected numeric feature count:', len(NUMERIC_FEATURES))
print('Sample numeric features:', NUMERIC_FEATURES[:20])
print('Cell 51 completed.')


--- Cell 51: Detect numeric features ---
Detected numeric feature count: 9
Sample numeric features: ['MONTH', 'DAY_OF_WEEK', 'SCHEDULED_DEPARTURE_HOUR', 'SCHEDULED_DEPARTURE_WEEKDAY', 'HOUR', 'HOUR_SIN', 'HOUR_COS', 'DOW_SIN', 'DOW_COS']
Cell 51 completed.


In [42]:
# Cell 52 – Detect categorical features
print('\n--- Cell 52: Detect categorical features ---')
# object and category dtypes
CAT_DTYPE = [c for c in FEATURE_LIST if pd.api.types.is_string_dtype(TRAIN_DF[c]) or pd.api.types.is_categorical_dtype(TRAIN_DF[c])]
# low-cardinality numeric may be categorical
LOW_CARD_THRESH = 50
CAT_FROM_NUM = [c for c in FEATURE_LIST if pd.api.types.is_numeric_dtype(TRAIN_DF[c]) and TRAIN_DF[c].nunique(dropna=True) <= LOW_CARD_THRESH]
CATEGORICAL_FEATURES = sorted(set(CAT_DTYPE + CAT_FROM_NUM))
print('Detected categorical feature count:', len(CATEGORICAL_FEATURES))
print('Sample categorical features:', CATEGORICAL_FEATURES[:20])
print('Cell 52 completed.')


--- Cell 52: Detect categorical features ---


C:\Users\Admin\AppData\Local\Temp\ipykernel_20728\416887090.py:4: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  CAT_DTYPE = [c for c in FEATURE_LIST if pd.api.types.is_string_dtype(TRAIN_DF[c]) or pd.api.types.is_categorical_dtype(TRAIN_DF[c])]


Detected categorical feature count: 18
Sample categorical features: ['CANCELLED', 'DAY', 'DAY_OF_WEEK', 'DISTANCE_KM', 'DIVERTED', 'DOW_COS', 'DOW_SIN', 'HOUR', 'HOUR_COS', 'HOUR_SIN', 'MONTH', 'ROUTE', 'SCHEDULED_ARRIVAL', 'SCHEDULED_DEPARTURE_HOUR', 'SCHEDULED_DEPARTURE_WEEKDAY', 'SOURCE_STATION', 'TRAIN_NUMBER', 'YEAR']
Cell 52 completed.


In [43]:
# Cell 53 – Detect label-encoded features
print('\n--- Cell 53: Detect label-encoded features ---')
LABEL_ENCODE_FEATURES = [c for c in FEATURE_LIST if pd.api.types.is_integer_dtype(TRAIN_DF[c]) and TRAIN_DF[c].nunique(dropna=True) <= 10]
print('Label-encoded candidates (<=10 unique ints):', LABEL_ENCODE_FEATURES)
print('Cell 53 completed.')


--- Cell 53: Detect label-encoded features ---
Label-encoded candidates (<=10 unique ints): []
Cell 53 completed.


In [44]:
# Cell 54 – Leakage check (DELAY_MINUTES)
print('\n--- Cell 54: Leakage check ---')
leakage_candidates = [c for c in TRAIN_DF.columns if 'DELAY' in c.upper() and c != TARGET_COL]
if leakage_candidates:
    print('Potential leakage columns (contain "DELAY" but are not target):', leakage_candidates)
else:
    print('No obvious delay leakage columns found')
print('Cell 54 completed.')


--- Cell 54: Leakage check ---
Potential leakage columns (contain "DELAY" but are not target): ['DELAY_DEPARTURE', 'SYSTEM_DELAY', 'SECURITY_DELAY', 'TRAIN_OPERATOR_DELAY', 'LATE_TRAIN_DELAY', 'WEATHER_DELAY', 'PREV_DELAY']
Cell 54 completed.


In [45]:
# Cell 55 – Numeric transformer
print('\n--- Cell 55: Numeric transformer ---')
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

SCALE_NUMERIC = False  # set True if you want StandardScaler for numeric features
numeric_steps = [('imputer', SimpleImputer(strategy='median'))]
if SCALE_NUMERIC:
    numeric_steps.append(('scaler', StandardScaler()))
NUMERIC_TRANSFORMER = Pipeline(steps=numeric_steps)
print('Numeric transformer ready; steps:', [s[0] for s in numeric_steps])
print('Cell 55 completed.')


--- Cell 55: Numeric transformer ---
Numeric transformer ready; steps: ['imputer']
Cell 55 completed.


In [ ]:
# Cell 56 – Categorical transformer
print('\n--- Cell 56: Categorical transformer ---')
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

# For high-cardinality categorical, use OrdinalEncoder to avoid explosion — else OneHot
CARDINALITY_THRESHOLD = 50
cat_ohe = [c for c in CATEGORICAL_FEATURES if TRAIN_DF[c].nunique(dropna=True) <= CARDINALITY_THRESHOLD]
cat_ord = [c for c in CATEGORICAL_FEATURES if TRAIN_DF[c].nunique(dropna=True) > CARDINALITY_THRESHOLD]

# instantiate OneHotEncoder compatibly across sklearn versions
try:
    ohe_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
except TypeError:
    ohe_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

cat_ohe_steps = [('imputer', SimpleImputer(strategy='most_frequent')), ('ohe', ohe_encoder)]
cat_ord_steps = [('imputer', SimpleImputer(strategy='most_frequent')), ('ord', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))]

CATEGORICAL_TRANSFORMER = {'ohe_cols': cat_ohe, 'ord_cols': cat_ord, 'ohe_pipeline': Pipeline(cat_ohe_steps), 'ord_pipeline': Pipeline(cat_ord_steps)}
print('Categorical OHE columns:', len(cat_ohe), 'Ordinal columns:', len(cat_ord))
print('Cell 56 completed.')


--- Cell 56: Categorical transformer ---


TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'

In [ ]:
# Cell 57 – ColumnTransformer (preprocessor)
print('\n--- Cell 57: Build ColumnTransformer preprocessor ---')
from sklearn.compose import ColumnTransformer

transformers = []
if len(NUMERIC_FEATURES) > 0:
    transformers.append(('num', NUMERIC_TRANSFORMER, NUMERIC_FEATURES))
if len(CATEGORICAL_TRANSFORMER['ohe_cols']) > 0:
    transformers.append(('cat_ohe', CATEGORICAL_TRANSFORMER['ohe_pipeline'], CATEGORICAL_TRANSFORMER['ohe_cols']))
if len(CATEGORICAL_TRANSFORMER['ord_cols']) > 0:
    transformers.append(('cat_ord', CATEGORICAL_TRANSFORMER['ord_pipeline'], CATEGORICAL_TRANSFORMER['ord_cols']))

if not transformers:
    raise ValueError('No transformers defined — check FEATURE_LIST and detected feature types')

PREPROCESSOR = ColumnTransformer(transformers=transformers, remainder='drop', verbose_feature_names_out=False)
print('Preprocessor created with transformers:', [t[0] for t in transformers])
print('Cell 57 completed.')

In [ ]:
# Cell 58 – Preprocessor sanity check
print('\n--- Cell 58: Preprocessor sanity check ---')
sample = X_train.sample(n=min(2000, len(X_train)), random_state=RANDOM_STATE)
try:
    PREPROCESSOR.fit(sample)
    out = PREPROCESSOR.transform(sample)
    print('Preprocessor fit/transform successful; transformed shape sample ->', out.shape)
except Exception as e:
    print('Preprocessor fit/transform FAILED:', e)

print('Cell 58 completed.')

In [ ]:
# Cell 59 – Feature count check
print('\n--- Cell 59: Feature count check ---')
try:
    transformed = PREPROCESSOR.transform(sample)
    if hasattr(transformed, 'shape'):
        print('Transformed feature count:', transformed.shape[1])
    else:
        print('Transformed output type:', type(transformed))
except Exception as e:
    print('Could not compute transformed feature count:', e)

print('Cell 59 completed.')

# Cell 60: End preprocessing section

End of Group F — preprocessing. Proceed to Group G (Split & CV) when ready.

# Group G — Split & CV (Cells 61–65)

Time-aware train/validation split, define TimeSeriesSplit, and perform a CV sanity check.

In [ ]:
# Cell 61 – Train / validation split (shuffle=False)
print('\n--- Cell 61: Train / validation split (time-aware) ---')
VAL_FRACTION = 0.2  # fraction for validation set
if TRAIN_DT_COL in TRAIN_DF.columns and TRAIN_DF[TRAIN_DT_COL].notna().sum() > 0:
    cutoff = TRAIN_DF[TRAIN_DT_COL].quantile(1 - VAL_FRACTION)
    train_mask = TRAIN_DF[TRAIN_DT_COL] <= cutoff
    val_mask = TRAIN_DF[TRAIN_DT_COL] > cutoff
    X_tr = X_train.loc[train_mask].reset_index(drop=True)
    y_tr = y_train.loc[train_mask].reset_index(drop=True)
    X_val = X_train.loc[val_mask].reset_index(drop=True)
    y_val = y_train.loc[val_mask].reset_index(drop=True)
    print(f"Time split by cutoff {cutoff} -> train: {len(X_tr)}, val: {len(X_val)}")
else:
    # fallback to simple index split
    split_idx = int(len(X_train) * (1 - VAL_FRACTION))
    X_tr = X_train.iloc[:split_idx].reset_index(drop=True)
    y_tr = y_train.iloc[:split_idx].reset_index(drop=True)
    X_val = X_train.iloc[split_idx:].reset_index(drop=True)
    y_val = y_train.iloc[split_idx:].reset_index(drop=True)
    print(f"Index split -> train: {len(X_tr)}, val: {len(X_val)}")

print('Cell 61 completed.')

In [ ]:
# Cell 62 – Print split sizes
print('\n--- Cell 62: Print split sizes ---')
print('X_tr shape:', X_tr.shape)
print('y_tr length:', len(y_tr))
print('X_val shape:', X_val.shape)
print('y_val length:', len(y_val))
print('Cell 62 completed.')

# Cell 63: TimeSeriesSplit definition (tscv)

Define a TimeSeriesSplit for time-aware CV and print split counts.

In [ ]:
# Cell 63 – TimeSeriesSplit definition (tscv)
from sklearn.model_selection import TimeSeriesSplit

N_SPLITS = 5
tscv = TimeSeriesSplit(n_splits=N_SPLITS)
print(f'Created TimeSeriesSplit with {N_SPLITS} splits')
print('Sample fold sizes (on training portion):')
for i, (tr_idx, val_idx) in enumerate(tscv.split(X_tr)):
    print(f'Fold {i+1}: train {len(tr_idx)}, val {len(val_idx)}')
    if i >= 4:
        break
print('Cell 63 completed.')

In [ ]:
# Cell 64 – CV sanity check
print('\n--- Cell 64: CV sanity check (quick pipeline run on first fold) ---')
from sklearn.linear_model import LinearRegression
try:
    # use first fold
    tr_idx, val_idx = next(iter(tscv.split(X_tr)))
    X_tr_fold = X_tr.iloc[tr_idx]
    y_tr_fold = y_tr.iloc[tr_idx]
    X_val_fold = X_tr.iloc[val_idx]
    y_val_fold = y_tr.iloc[val_idx]

    lr_sm = pipeline.Pipeline([('preprocessor', PREPROCESSOR), ('lr', LinearRegression())])
    lr_sm.fit(X_tr_fold, y_tr_fold)
    preds = lr_sm.predict(X_val_fold)
    m = metrics_summary_regression(y_val_fold, preds)
    print('Fold 1 quick LR metrics:', m)
except Exception as e:
    print('CV sanity check failed:', e)

print('Cell 64 completed.')

# Cell 65: End split section

End of Group G — split & CV. Proceed to Group H (Baseline models) when ready.

# Group H — Baseline Models (Cells 66–70)

Simple baselines: Linear regression and Random Forest, evaluate on validation set, save baseline results.

In [ ]:
# Cell 66 – Linear regression baseline
print('\n--- Cell 66: Linear regression baseline ---')
from sklearn.linear_model import LinearRegression

lr_pipe = pipeline.Pipeline([('preprocessor', PREPROCESSOR), ('lr', LinearRegression())])
lr_pipe.fit(X_tr, y_tr)
lr_preds = lr_pipe.predict(X_val)
lr_metrics = metrics_summary_regression(y_val, lr_preds)
print('Linear Regression metrics on validation:', lr_metrics)

print('Cell 66 completed.')

In [ ]:
# Cell 67 – Random Forest baseline
print('\n--- Cell 67: Random Forest baseline ---')
rf_pipe = pipeline.Pipeline([('preprocessor', PREPROCESSOR), ('rf', RandomForestRegressor(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1))])
rf_pipe.fit(X_tr, y_tr)
rf_preds = rf_pipe.predict(X_val)
rf_metrics = metrics_summary_regression(y_val, rf_preds)
print('Random Forest metrics on validation:', rf_metrics)

print('Cell 67 completed.')

In [ ]:
# Cell 68 – Baseline evaluation
print('\n--- Cell 68: Baseline evaluation ---')
baseline_results = []
baseline_results.append({'model': 'median_baseline', **metrics_summary_regression(y_val, np.repeat(y_tr.median(), len(y_val)))})
baseline_results.append({'model': 'linear_regression', **lr_metrics})
baseline_results.append({'model': 'random_forest', **rf_metrics})

baseline_df = pd.DataFrame(baseline_results).set_index('model')
print('\nBaseline comparison:')
print(baseline_df)

print('Cell 68 completed.')

In [ ]:
# Cell 69 – Save baseline results
print('\n--- Cell 69: Save baseline results ---')
OUT_REPORTS = 'reports'
os.makedirs(OUT_REPORTS, exist_ok=True)
baseline_path = os.path.join(OUT_REPORTS, 'baseline_results.csv')
try:
    baseline_df.to_csv(baseline_path)
    print('Saved baseline results to', baseline_path)
except Exception as e:
    print('Could not save baseline results:', e)

print('Cell 69 completed.')

# Cell 70: End baseline section

End of Group H — Baseline Models. Proceed to Group I (Tuning & Advanced Models) when ready.

# Group I — Tuning & Advanced Models (Cells 71–78)

Set up Optuna objective, parameter space, create study, run optimization, summarize best params, train tuned model, validate and compute metrics.

In [ ]:
# Cell 71 – Define Optuna objective
print('\n--- Cell 71: Define Optuna objective (LGBM) ---')
import optuna
from optuna.samplers import TPESampler
from lightgbm import LGBMRegressor

N_TRIALS = 20  # default; adjust as needed

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 16, 128),
        'max_depth': trial.suggest_int('max_depth', -1, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 200),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0)
    }

    # time-series CV using tscv
    rmses = []
    for tr_idx, val_idx in tscv.split(X_tr):
        X_tr_fold = X_tr.iloc[tr_idx]
        y_tr_fold = y_tr.iloc[tr_idx]
        X_val_fold = X_tr.iloc[val_idx]
        y_val_fold = y_tr.iloc[val_idx]

        pipe = pipeline.Pipeline([
            ('preprocessor', PREPROCESSOR),
            ('model', LGBMRegressor(random_state=RANDOM_STATE, **params))
        ])
        pipe.fit(X_tr_fold, y_tr_fold)
        preds = pipe.predict(X_val_fold)
        rmse = mean_squared_error(y_val_fold, preds, squared=False)
        rmses.append(rmse)
    return float(np.mean(rmses))

print('Objective defined. Adjust N_TRIALS and run Cell 74 to optimize.')

In [ ]:
# Cell 72 – Define parameter space (documented)
print('\n--- Cell 72: Parameter space (documented) ---')
print('Using LGBM parameter search: n_estimators, learning_rate, num_leaves, max_depth, min_child_samples, subsample, colsample_bytree, reg_alpha, reg_lambda')
print('You can customize ranges in Cell 71 (objective).')
print('Cell 72 completed.')

In [ ]:
# Cell 73 – Create Optuna study
print('\n--- Cell 73: Create Optuna study ---')
study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=RANDOM_STATE))
print('Study created. Run Cell 74 to start optimization (this may take time depending on N_TRIALS).')
print('Cell 73 completed.')

In [ ]:
# Cell 74 – Run Optuna optimization
print('\n--- Cell 74: Run Optuna optimization ---')
# WARNING: may take time. Adjust N_TRIALS to a small value for quick runs.
N_TRIALS = 20
try:
    study.optimize(objective, n_trials=N_TRIALS)
    print('Optimization completed')
except Exception as e:
    print('Optuna optimization failed or was interrupted:', e)

print('Cell 74 completed.')

In [ ]:
# Cell 75 – Best params summary
print('\n--- Cell 75: Best params summary ---')
try:
    print('Best value (RMSE):', study.best_value)
    print('Best params:')
    print(study.best_params)
    BEST_PARAMS = study.best_params
except Exception as e:
    print('Could not read study results:', e)
    BEST_PARAMS = None

print('Cell 75 completed.')

In [ ]:
# Cell 76 – Train tuned model
print('\n--- Cell 76: Train tuned model with best params ---')
if BEST_PARAMS is not None:
    try:
        tuned_pipe = pipeline.Pipeline([
            ('preprocessor', PREPROCESSOR),
            ('model', LGBMRegressor(random_state=RANDOM_STATE, **BEST_PARAMS))
        ])
        tuned_pipe.fit(X_tr, y_tr)
        print('Tuned model trained on training portion.')
    except Exception as e:
        print('Training tuned model failed:', e)
        tuned_pipe = None
else:
    print('BEST_PARAMS is None — cannot train tuned model')
    tuned_pipe = None

print('Cell 76 completed.')

In [ ]:
# Cell 77 – Validation prediction
print('\n--- Cell 77: Validation prediction with tuned model ---')
if tuned_pipe is not None:
    val_preds_tuned = tuned_pipe.predict(X_val)
    print('Predicted on validation set; sample predictions:')
    print(val_preds_tuned[:5])
else:
    print('Tuned model not available; skipping predictions')

print('Cell 77 completed.')

In [ ]:
# Cell 78 – Metrics calculation for tuned model
print('\n--- Cell 78: Metrics calculation for tuned model ---')
if tuned_pipe is not None:
    tuned_metrics = metrics_summary_regression(y_val, val_preds_tuned)
    print('Tuned model metrics on validation:', tuned_metrics)
    # append to baseline_df for comparison
    try:
        baseline_df.loc['tuned_lgbm'] = tuned_metrics
    except Exception:
        baseline_df = baseline_df.append(pd.Series(tuned_metrics, name='tuned_lgbm'))
else:
    print('Tuned model not available; skipping metrics calculation')

print('Cell 78 completed.')

# Group J — Finalization (Cells 79–82)

Compare models, inspect feature importance or SHAP, save the best model, and run inference on the test set (if available).

In [ ]:
# Cell 79 – Model comparison table
print('\n--- Cell 79: Model comparison table ---')
try:
    print(baseline_df.sort_values('rmse'))
except Exception as e:
    print('Could not display baseline_df:', e)

print('Cell 79 completed.')

In [ ]:
# Cell 80 – Feature importance / SHAP
print('\n--- Cell 80: Feature importance / SHAP ---')
if tuned_pipe is not None:
    try:
        model = tuned_pipe.named_steps['model']
        # try to get feature names
        try:
            feat_names = PREPROCESSOR.get_feature_names_out(FEATURE_LIST)
        except Exception:
            try:
                feat_names = PREPROCESSOR.get_feature_names_out()
            except Exception:
                feat_names = [f'f{i}' for i in range(model.feature_importances_.shape[0])]

        importances = model.feature_importances_
        fi = pd.DataFrame({'feature': feat_names, 'importance': importances})
        fi = fi.sort_values('importance', ascending=False).head(30)
        print('Top feature importances:')
        print(fi)
    except Exception as e:
        print('Could not compute feature importances:', e)
    # SHAP (optional)
    try:
        import shap
        explainer = shap.TreeExplainer(model)
        # sample for SHAP due to cost
        sample_X = X_val.sample(n=min(500, len(X_val)), random_state=RANDOM_STATE)
        X_trans = PREPROCESSOR.transform(sample_X)
        shap_values = explainer.shap_values(X_trans)
        print('SHAP computed for sample; consider plotting (not shown here).')
    except Exception as e:
        print('SHAP not available or failed:', e)
else:
    print('tuned_pipe not available; skipping feature importance')

print('Cell 80 completed.')

In [ ]:
# Cell 81 – Save best model (joblib.dump)
print('\n--- Cell 81: Save best model ---')
if tuned_pipe is not None:
    out_path = os.path.join(MODEL_DIR, 'best_model.joblib')
    try:
        joblib.dump(tuned_pipe, out_path)
        print('Saved best model to', out_path)
    except Exception as e:
        print('Could not save model:', e)
else:
    print('No tuned model to save')

print('Cell 81 completed.')

In [ ]:
# Cell 82 – Inference on test / end notebook
print('\n--- Cell 82: Inference on test (if available) ---')
if TEST_DF is not None and 'PREV_DELAY' in TEST_DF.columns:
    X_test = TEST_DF[FEATURE_LIST].copy()
    if tuned_pipe is not None:
        try:
            pred_test = tuned_pipe.predict(X_test)
            out_df = TEST_DF.copy()
            out_df['PRED_DELAY'] = pred_test
            out_path = os.path.join('reports', 'test_predictions.csv')
            out_df.to_csv(out_path, index=False)
            print('Saved test predictions to', out_path)
        except Exception as e:
            print('Could not run inference on test:', e)
    else:
        print('No tuned model; skipping test inference')
else:
    print('No test set available or required features missing; skipping inference')

print('Cell 82 completed — END of notebook pipeline.')